In [1]:
from google import colab
colab.drive.mount("/content/drive")

Mounted at /content/drive


In [1]:
! pip install timm

In [2]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

from sklearn.model_selection import StratifiedKFold
device = torch.device('cuda')

In [3]:
path = '/home/'

In [4]:
train_png = sorted(glob(path + 'open/train/*.png'))
test_png = sorted(glob(path + 'open/test/*.png'))

In [5]:
len(train_png), len(test_png)

(4277, 2154)

In [6]:
train_y = pd.read_csv(path +"open/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [7]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (384, 384),interpolation = cv2.INTER_AREA)
    return img

In [8]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|██████████| 2154/2154 [01:24<00:00, 25.58it/s]


In [9]:
np.save(path + 'train_imgs_384', np.array(train_imgs))
np.save(path + 'test_imgs_384', np.array(test_imgs))

In [10]:
train_imgs = np.load(path + 'train_imgs_384.npy')
test_imgs = np.load(path + 'test_imgs_384.npy')

In [11]:
meanRGB = [np.mean(x, axis=(0,1)) for x in train_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in train_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("train 평균",meanR, meanG, meanB)
print("train 표준편차",stdR, stdG, stdB)

train 평균 0.4330380901867049 0.4034575319032911 0.39415050509784405
train 표준편차 0.1815717110252788 0.17403455556798705 0.16323395055036488


In [12]:
meanRGB = [np.mean(x, axis=(0,1)) for x in test_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in test_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("test 평균",meanR, meanG, meanB)
print("test 표준편차",stdR, stdG, stdB)

test 평균 0.41825619520929724 0.3931011906330291 0.386631764639131
test 표준편차 0.19505524270747931 0.19005280951759498 0.18053225852732663


In [13]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
          train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.433038, 0.403458, 0.394151],
                                     std = [0.181572, 0.174035, 0.163234]),
                transforms.RandomAffine((-45, 45)),
                
            ])
          img = train_transform(img)
        if self.mode == 'test':
          test_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.418256, 0.393101, 0.386632],
                                     std = [0.195055, 0.190053, 0.185323])
            ])
          img = test_transform(img)

        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
          self.model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=88, drop_path_rate = 0.2)
        if self.mode == 'test':
          self.model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=88, drop_path_rate = 0)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [14]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [15]:
def main(seed = 2022):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    
main(2022)

In [16]:
import gc

cv = StratifiedKFold(n_splits = 5, random_state = 2022,shuffle=True)
batch_size = 32
epochs = 70
pred_ensemble = []


for idx, (train_idx, val_idx) in enumerate(cv.split(train_imgs, np.array(train_labels))):
  print("----------fold_{} start!----------".format(idx))
  t_imgs, val_imgs = train_imgs[train_idx],  train_imgs[val_idx]
  t_labels, val_labels = np.array(train_labels)[train_idx], np.array(train_labels)[val_idx]

  # Train
  train_dataset = Custom_dataset(np.array(t_imgs), np.array(t_labels), mode='train')
  train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

  # Val
  val_dataset = Custom_dataset(np.array(val_imgs), np.array(val_labels), mode='test')
  val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

  gc.collect()
  torch.cuda.empty_cache()
  best=0

  model = Network().to(device)

  optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay = 2e-2)
  criterion = nn.CrossEntropyLoss()
  scaler = torch.cuda.amp.GradScaler()  

  best_f1 = 0
  early_stopping = 0
  for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
    train_f1 = score_function(train_y, train_pred)
    state_dict= model.state_dict()
    model.eval()
    with torch.no_grad():
      val_loss = 0 
      val_pred = []
      val_y = []
      

      for batch in (val_loader):
        x_val = torch.tensor(batch[0], dtype = torch.float32, device = device)
        y_val = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred_val = model(x_val)
        loss_val = criterion(pred_val, y_val)

        val_loss += loss_val.item()/len(val_loader)
        val_pred += pred_val.argmax(1).detach().cpu().numpy().tolist()
        val_y += y_val.detach().cpu().numpy().tolist()
      val_f1 = score_function(val_y, val_pred)

      if val_f1 > best_f1:
        best_epoch = epoch
        best_loss = val_loss
        best_f1 = val_f1
        early_stopping = 0

        torch.save({'epoch':epoch,
                    'state_dict':state_dict,
                    'optimizer': optimizer.state_dict(),
                    'scaler': scaler.state_dict(),
             }, path +'best_model_{}.pth'.format(idx))
        print('-----------------SAVE:{} epoch----------------'.format(best_epoch+1))
      else:
          early_stopping += 1

            # Early Stopping
      if early_stopping == 20:
        TIME = time.time() - start
        print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
        print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
        print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')
        break

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')

----------fold_0 start!----------


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b3_ra2-cf984f9c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_ra2-cf984f9c.pth


-----------------SAVE:1 epoch----------------
epoch : 1/70    time : 47s/3256s
TRAIN    loss : 1.18003    f1 : 0.17458
Val    loss : 0.67017    f1 : 0.24356
-----------------SAVE:2 epoch----------------
epoch : 2/70    time : 42s/2850s
TRAIN    loss : 0.54855    f1 : 0.34165
Val    loss : 0.49407    f1 : 0.39124
-----------------SAVE:3 epoch----------------
epoch : 3/70    time : 42s/2783s
TRAIN    loss : 0.38202    f1 : 0.50511
Val    loss : 0.37423    f1 : 0.53545
-----------------SAVE:4 epoch----------------
epoch : 4/70    time : 45s/2968s
TRAIN    loss : 0.28057    f1 : 0.59147
Val    loss : 0.31233    f1 : 0.62464
-----------------SAVE:5 epoch----------------
epoch : 5/70    time : 43s/2774s
TRAIN    loss : 0.20686    f1 : 0.72829
Val    loss : 0.25508    f1 : 0.64506
-----------------SAVE:6 epoch----------------
epoch : 6/70    time : 40s/2589s
TRAIN    loss : 0.16698    f1 : 0.78842
Val    loss : 0.22048    f1 : 0.70016
-----------------SAVE:7 epoch----------------
epoch : 7/70

-----------------SAVE:2 epoch----------------
epoch : 2/70    time : 41s/2756s
TRAIN    loss : 0.52734    f1 : 0.34787
Val    loss : 0.42744    f1 : 0.41935
-----------------SAVE:3 epoch----------------
epoch : 3/70    time : 40s/2699s
TRAIN    loss : 0.37642    f1 : 0.52579
Val    loss : 0.36167    f1 : 0.53129
-----------------SAVE:4 epoch----------------
epoch : 4/70    time : 42s/2791s
TRAIN    loss : 0.27608    f1 : 0.63821
Val    loss : 0.24869    f1 : 0.64042
-----------------SAVE:5 epoch----------------
epoch : 5/70    time : 41s/2686s
TRAIN    loss : 0.20599    f1 : 0.70789
Val    loss : 0.19019    f1 : 0.66433
-----------------SAVE:6 epoch----------------
epoch : 6/70    time : 41s/2596s
TRAIN    loss : 0.15448    f1 : 0.80644
Val    loss : 0.19303    f1 : 0.72520
-----------------SAVE:7 epoch----------------
epoch : 7/70    time : 41s/2584s
TRAIN    loss : 0.11675    f1 : 0.85582
Val    loss : 0.18720    f1 : 0.77551
epoch : 8/70    time : 41s/2559s
TRAIN    loss : 0.09406  

epoch : 69/70    time : 39s/39s
TRAIN    loss : 0.00720    f1 : 0.99337
Val    loss : 0.21583    f1 : 0.78208
epoch : 70/70    time : 40s/0s
TRAIN    loss : 0.00690    f1 : 0.99157
Val    loss : 0.20141    f1 : 0.81555
----------fold_2 start!----------
-----------------SAVE:1 epoch----------------
epoch : 1/70    time : 45s/3086s
TRAIN    loss : 1.21485    f1 : 0.17259
Val    loss : 0.62505    f1 : 0.24639
-----------------SAVE:2 epoch----------------
epoch : 2/70    time : 40s/2731s
TRAIN    loss : 0.54730    f1 : 0.33472
Val    loss : 0.42973    f1 : 0.37020
-----------------SAVE:3 epoch----------------
epoch : 3/70    time : 40s/2680s
TRAIN    loss : 0.36599    f1 : 0.52247
Val    loss : 0.27733    f1 : 0.55605
-----------------SAVE:4 epoch----------------
epoch : 4/70    time : 40s/2662s
TRAIN    loss : 0.26594    f1 : 0.65265
Val    loss : 0.23799    f1 : 0.59698
-----------------SAVE:5 epoch----------------
epoch : 5/70    time : 42s/2736s
TRAIN    loss : 0.19550    f1 : 0.73906


-----------------SAVE:6 epoch----------------
epoch : 6/70    time : 40s/2587s
TRAIN    loss : 0.13485    f1 : 0.82656
Val    loss : 0.18239    f1 : 0.73501
-----------------SAVE:7 epoch----------------
epoch : 7/70    time : 40s/2528s
TRAIN    loss : 0.11254    f1 : 0.86498
Val    loss : 0.15266    f1 : 0.75473
-----------------SAVE:8 epoch----------------
epoch : 8/70    time : 41s/2539s
TRAIN    loss : 0.09184    f1 : 0.88427
Val    loss : 0.18032    f1 : 0.76833
epoch : 9/70    time : 42s/2545s
TRAIN    loss : 0.07311    f1 : 0.90049
Val    loss : 0.18976    f1 : 0.74187
-----------------SAVE:10 epoch----------------
epoch : 10/70    time : 40s/2410s
TRAIN    loss : 0.06178    f1 : 0.92467
Val    loss : 0.16073    f1 : 0.77272
-----------------SAVE:11 epoch----------------
epoch : 11/70    time : 41s/2413s
TRAIN    loss : 0.06141    f1 : 0.93477
Val    loss : 0.14852    f1 : 0.78937
epoch : 12/70    time : 40s/2325s
TRAIN    loss : 0.04679    f1 : 0.95763
Val    loss : 0.16440    f

epoch : 17/70    time : 40s/2122s
TRAIN    loss : 0.02000    f1 : 0.98277
Val    loss : 0.15462    f1 : 0.79483
epoch : 18/70    time : 40s/2101s
TRAIN    loss : 0.02685    f1 : 0.96939
Val    loss : 0.19076    f1 : 0.78083
epoch : 19/70    time : 41s/2089s
TRAIN    loss : 0.02669    f1 : 0.97793
Val    loss : 0.17328    f1 : 0.77480
-----------------SAVE:20 epoch----------------
epoch : 20/70    time : 42s/2088s
TRAIN    loss : 0.03179    f1 : 0.96624
Val    loss : 0.18530    f1 : 0.80036
epoch : 21/70    time : 40s/1980s
TRAIN    loss : 0.02341    f1 : 0.97496
Val    loss : 0.19689    f1 : 0.78856
epoch : 22/70    time : 40s/1944s
TRAIN    loss : 0.02628    f1 : 0.97106
Val    loss : 0.17547    f1 : 0.76873
epoch : 23/70    time : 41s/1915s
TRAIN    loss : 0.03657    f1 : 0.95278
Val    loss : 0.19700    f1 : 0.77782
epoch : 24/70    time : 42s/1930s
TRAIN    loss : 0.02569    f1 : 0.97535
Val    loss : 0.17588    f1 : 0.78824
-----------------SAVE:25 epoch----------------
epoch : 25

In [17]:
pred_ensemble = []
batch_size = 32
# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

for i in range(5):
  model_test = Network(mode = 'test').to(device)
  model_test.load_state_dict(torch.load((path+'best_model_{}.pth'.format(i)))['state_dict'])
  model_test.eval()
  pred_prob = []
  with torch.no_grad():
      for batch in (test_loader):
          x = torch.tensor(batch[0], dtype = torch.float32, device = device)
          with torch.cuda.amp.autocast():
              pred = model_test(x)
              pred_prob.extend(pred.detach().cpu().numpy())
      pred_ensemble.append(pred_prob)

In [18]:
len(pred_ensemble)

5

In [19]:
pred = (np.array(pred_ensemble[0])+ np.array(pred_ensemble[1])+ np.array(pred_ensemble[3]) + np.array(pred_ensemble[4]) )/4
f_pred = np.array(pred).argmax(1).tolist()

In [20]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [21]:
submission = pd.read_csv(path + "open/sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [22]:
submission.to_csv(path + "b3_norm_epoch70_4_2.csv", index = False)

### 모델 학습

사전 학습 모델의 성능 파악을 할 때 Fold 학습은 실행 시간이 오래걸려서 fold를 나누지 않은 데이터에 대해서 학습을 진행하고 성능을 비교하였습니다.

In [23]:
batch_size = 32
epochs = 30

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [24]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [25]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

model = Network().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay = 1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 

batch_size = 32
epochs = 30

best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

epoch : 1/30    time : 52s/1495s
TRAIN    loss : 1.62875    f1 : 0.15265
epoch : 2/30    time : 47s/1312s
TRAIN    loss : 0.78266    f1 : 0.19686
epoch : 3/30    time : 48s/1294s
TRAIN    loss : 0.59936    f1 : 0.29615
epoch : 4/30    time : 46s/1209s
TRAIN    loss : 0.47578    f1 : 0.39296
epoch : 5/30    time : 49s/1224s
TRAIN    loss : 0.40606    f1 : 0.48568
epoch : 6/30    time : 49s/1164s
TRAIN    loss : 0.34266    f1 : 0.55320
epoch : 7/30    time : 48s/1104s
TRAIN    loss : 0.28676    f1 : 0.60903
epoch : 8/30    time : 48s/1049s
TRAIN    loss : 0.24467    f1 : 0.69303
epoch : 9/30    time : 49s/1037s
TRAIN    loss : 0.20959    f1 : 0.71534
epoch : 10/30    time : 46s/922s
TRAIN    loss : 0.18401    f1 : 0.77599
epoch : 11/30    time : 47s/893s
TRAIN    loss : 0.15921    f1 : 0.81108
epoch : 12/30    time : 49s/877s
TRAIN    loss : 0.13784    f1 : 0.83371
epoch : 13/30    time : 49s/826s
TRAIN    loss : 0.12121    f1 : 0.84938
epoch : 14/30    time : 46s/741s
TRAIN    loss : 0.

### 추론

In [26]:
model.eval()
f_pred = []
pred_prob = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
            pred_prob.extend(pred.detach().cpu().numpy())
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [27]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

### 제출물 생성

In [28]:
submission = pd.read_csv(path + "open/sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [29]:
submission.to_csv(path + " b3_norm.csv", index = False)